In [2]:
from google.colab import files
uoloded=files.upload();

Saving  Superstore.csv to  Superstore.csv


In [12]:
import pandas as pd
df=pd.read_csv("/content/drive/MyDrive/Global Sales Performance Analytics Dashboard/ Superstore.csv",parse_dates=["Order Date"])
df.head()

df.info()
df.describe()
df.isna().sum()
df.shape


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 22 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   s              9994 non-null   int64         
 1   Order ID       9994 non-null   object        
 2   Order Date     9994 non-null   datetime64[ns]
 3   Ship Date      9994 non-null   object        
 4   Ship Mode      9994 non-null   object        
 5   Customer ID    9994 non-null   object        
 6   Customer Name  9994 non-null   object        
 7   Segment        9994 non-null   object        
 8   Country        9994 non-null   object        
 9   City           9994 non-null   object        
 10  State          9994 non-null   object        
 11  Postal Code    9994 non-null   int64         
 12  Region         9994 non-null   object        
 13  Product ID     9994 non-null   object        
 14  Category       9994 non-null   object        
 15  Sub-Category   9994 n

(9994, 22)

####Data Cleaning

In [19]:
# Remove duplicate rows
df = df.drop_duplicates()

# column to numeric
df['Sales'] = pd.to_numeric(df['Sales'], errors='coerce')
df['Quantity'] = pd.to_numeric(df['Quantity'], errors='coerce')
df['Profit'] = pd.to_numeric(df['Profit'], errors='coerce')
df['Discount'] = pd.to_numeric(df['Discount'], errors='coerce')



# Fill missing quantity with 1 (safe default)
df['Quantity'] = df['Quantity'].fillna(1).astype(int)

# Fill missing Sales with median
df['Sales'] = df['Sales'].fillna(df['Sales'].median())

# Fill missing Profit with 0 (safe default)
df['Profit'] = df['Profit'].fillna(0)

# Fill missing Discount with 0
df['Discount'] = df['Discount'].fillna(0)

In [20]:
df.isna().sum()


,0
s,0
Order ID,0
Order Date,0
Ship Date,0
Ship Mode,0
Customer ID,0
Customer Name,0
Segment,0
Country,0
City,0


In [22]:
# 1) Convert Order Date to datetime if not already done
df['Order Date'] = pd.to_datetime(df['Order Date'])

# 2) Create Month and Year columns
df['Year'] = df['Order Date'].dt.year
df['Month'] = df['Order Date'].dt.to_period('M').dt.to_timestamp()

# 3) Ensure numeric columns are clean
df['Sales'] = pd.to_numeric(df['Sales'], errors='coerce').fillna(0)
df['Profit'] = pd.to_numeric(df['Profit'], errors='coerce').fillna(0)
df['Quantity'] = pd.to_numeric(df['Quantity'], errors='coerce').fillna(1).astype(int)
df['Discount'] = pd.to_numeric(df['Discount'], errors='coerce').fillna(0)

# 4) Calculate unit price (approx)
df['UnitPrice'] = df['Sales'] / df['Quantity']

# 5) Calculate price after discount
df['PriceAfterDiscount'] = df['UnitPrice'] * (1 - df['Discount'])

# 6) Profit Margin % (avoid division by zero)
df['ProfitMargin'] = df.apply(
    lambda row: (row['Profit'] / row['Sales']) if row['Sales'] != 0 else 0,
    axis=1
)

# 7) Show updated dataframe
df.head()


,s,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Profit,Unnamed: 21,Year,Month,unit_price,after_discount_price,profit_margin,UnitPrice,PriceAfterDiscount,ProfitMargin
0,1,CA-2016-152156,2016-11-08,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,41.9136,NaN,2016,2016-11-01,130.9800,130.980000,0.1600,130.9800,130.980000,0.1600
1,2,CA-2016-152156,2016-11-08,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,219.5820,NaN,2016,2016-11-01,243.9800,243.980000,0.3000,243.9800,243.980000,0.3000
2,3,CA-2016-138688,2016-06-12,6/16/2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,6.8714,NaN,2016,2016-06-01,7.3100,7.310000,0.4700,7.3100,7.310000,0.4700
3,4,US-2015-108966,2015-10-11,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,-383.0310,NaN,2015,2015-10-01,191.5155,105.333525,-0.4000,191.5155,105.333525,-0.4000
4,5,US-2015-108966,2015-10-11,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,2.5164,NaN,2015,2015-10-01,11.1840,8.947200,0.1125,11.1840,8.947200,0.1125


In [23]:
df[['Sales','Profit','Quantity','Discount','UnitPrice','ProfitMargin']].head()


,Sales,Profit,Quantity,Discount,UnitPrice,ProfitMargin
0,261.9600,41.9136,2,0.00,130.9800,0.1600
1,731.9400,219.5820,3,0.00,243.9800,0.3000
2,14.6200,6.8714,2,0.00,7.3100,0.4700
3,957.5775,-383.0310,5,0.45,191.5155,-0.4000
4,22.3680,2.5164,2,0.20,11.1840,0.1125


##Monthly Revenue Trend Chart


In [29]:
import plotly.express as px

monthly = df.groupby('Month')['Sales'].sum().reset_index()

fig = px.line(
    monthly,
    x='Month',
    y='Sales',
    title='📈 Monthly Revenue Trend',
    markers=True
)

fig.show()


##Category-wise Revenue


In [30]:
category = df.groupby('Category')['Sales'].sum().reset_index().sort_values('Sales', ascending=False)

fig = px.bar(
    category,
    x='Category',
    y='Sales',
    title='📊 Revenue by Category',
    color='Category'
)

fig.show()


##Region-wise Revenue

In [32]:
region = df.groupby('Region')['Sales'].sum().reset_index().sort_values('Sales', ascending=False)

fig = px.bar(
    region,
    x='Region',
    y='Sales',
    title='🌍 Revenue by Region',
    color='Region'
)

fig.show()


##Top 10 Customers

In [33]:
top_customers = df.groupby('Customer ID')['Sales'].sum().reset_index().sort_values('Sales', ascending=False).head(10)

fig = px.bar(
    top_customers,
    x='Customer ID',
    y='Sales',
    title='🏆 Top 10 Customers by Sales',
    color='Sales'
)

fig.show()
